# Zillow Clustering Project Scratchpad

In [1]:
import pandas as pd
import numpy as np
import acquire

In [3]:
# can I bring in the appropriate data?
df = acquire.get_zillow_cluster()
acquire.overview(df)

This dataframe has 77574 rows and 68 columns.
----------------------------------

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77574 entries, 0 to 77573
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   heatingorsystemtypeid         49569 non-null  float64
 1   buildingclasstypeid           15 non-null     float64
 2   architecturalstyletypeid      206 non-null    float64
 3   airconditioningtypeid         25006 non-null  float64
 4   parcelid                      77574 non-null  int64  
 5   typeconstructiontypeid        222 non-null    float64
 6   storytypeid                   50 non-null     float64
 7   propertylandusetypeid         77574 non-null  float64
 8   id                            77574 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77574 non-null  float64
 11  bedroomcnt                    77574 n

#### Data Aquistion Key Findings & Takeaways:

- This dataframe has 77574 rows and 68 columns.
- Target variable: 'logerror'
- There are 11 columns that are dtype "object" that either need to be dropped, or converted to int
- There are numerous columns with mostly missing values that either need to be dropped, or imputed
- There are numerous columns that at first glance appear to be duplicates, or similar enough to either be dropped, or joined